# Defining a Delayed Flight

### 1. A delayed flight will be a flight that arrives late at its destination
### 2. If the flight has any delays from its departure, but still arrives at its destination on time, it will not be considered a delayed.
### Furthermore, this will be handled as a binary classification problem, with a 0 for arrival on time and a 1 for delayed arrival

# Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
pd.set_option('display.max_columns', None)
import datetime, warnings, scipy
warnings.filterwarnings("ignore")

# Data Loading 

In [3]:
df1 = pd.read_csv("C:\\Users\\udaya\\OneDrive\\Documents\\archive\\2018.csv")
df1.head(2).append(df1.tail(2))

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY,Unnamed: 27
0,2018-01-01,UA,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,NaN,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,UA,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,NaN,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN,NaN
7213444,2018-12-31,AA,1818,CLT,RDU,1300,1323.0,23.0,11.0,1334.0,1400.0,4.0,1350,1404.0,14.0,0.0,NaN,0.0,50.0,41.0,26.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN
7213445,2018-12-31,AA,1818,RDU,CLT,1435,1443.0,8.0,8.0,1451.0,1535.0,7.0,1546,1542.0,-4.0,0.0,NaN,0.0,71.0,59.0,44.0,130.0,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
df1.shape

(7213446, 28)

Looking for the details of each feature, finding missing values and anything that we will not be using

In [5]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7213446 entries, 0 to 7213445
Data columns (total 28 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   OP_CARRIER_FL_NUM    int64  
 3   ORIGIN               object 
 4   DEST                 object 
 5   CRS_DEP_TIME         int64  
 6   DEP_TIME             float64
 7   DEP_DELAY            float64
 8   TAXI_OUT             float64
 9   WHEELS_OFF           float64
 10  WHEELS_ON            float64
 11  TAXI_IN              float64
 12  CRS_ARR_TIME         int64  
 13  ARR_TIME             float64
 14  ARR_DELAY            float64
 15  CANCELLED            float64
 16  CANCELLATION_CODE    object 
 17  DIVERTED             float64
 18  CRS_ELAPSED_TIME     float64
 19  ACTUAL_ELAPSED_TIME  float64
 20  AIR_TIME             float64
 21  DISTANCE             float64
 22  CARRIER_DELAY        float64
 23  WEATHER_DELAY        float64
 24

# Glossary

<b>FL_DATE</b> = Date of the Flight<br>
<b>OP_CARRIER</b> = Airline Identifier<br>
<b>OP_CARRIER_FL_NUM</b> = Flight Number<br>
<b>ORIGIN</b> = Starting Airport Code<br>
<b>DEST</b> = Destination Airport Code<br>
<b>CRS_DEP_TIME</b> = Planned Departure Time<br>
<b>DEP_TIME</b> = Actual Departure Time<br>
<b>DEP_DELAY</b> = Total Delay on Departure in minutes<br>
<b>TAXI_OUT</b> = The time duration elapsed between departure from the origin airport gate and wheels off<br>
<b>WHEELS_OFF</b> = The time point that the aircraft's wheels leave the ground<br>
<b>WHEELS_ON</b> = The time point that the aircraft'ss wheels touch on the ground<br>
<b>TAXI_IN</b> = The time duration elapsed between wheels-on and gate arrival at the destination airport<br>
<b>CRS_ARR_TIME</b> = Planned arrival time<br>
<b>ARR_TIME</b> = Actual Arrival Time = ARRIVAL_TIME - SCHEDULED_ARRIVAL<br>
<b>ARR_DELAY</b> = Total Delay on Arrival in minutes<br>
<b>CANCELLED</b> = Flight Cancelled (1 = cancelled)<br>
<b>CANCELLATION_CODE</b> = Reason for Cancellation of flight: A - Airline/Carrier; B - Weather; C - National Air System; D - Security<br>
<b>DIVERTED</b> = Aircraft landed on different airport that the one scheduled<br>
<b>CRS_ELAPSED_TIME</b> = Planned time amount needed for the flight trip<br>
<b>ACTUAL_ELAPSED_TIME</b> = AIR_TIME+TAXI_IN+TAXI_OUT<br>
<b>AIR_TIME</b> = The time duration between wheels_off and wheels_on time<br>
<b>DISTANCE</b> = Distance between two airports<br>
<b>CARRIER_DELAY</b> = Delay caused by the airline in minutes<br>
<b>WEATHER_DELAY</b> = Delay caused by weather<br>
<b>NAS_DELAY</b> = Delay caused by air system<br>
<b>SECURITY_DELAY</b> = caused by security reasons<br>
<b>LATE_AIRCRAFT_DELAY</b> = Delay caused by security<br>

Source: MetaData



# Data Preprocessing

### OP_CARRIER COLUMN

This column refers to the airline identifier. We will change these identifiers to their extended names. The source used to find the extended names is wikipedia

In [6]:
df1.OP_CARRIER.unique()

array(['UA', 'AS', '9E', 'B6', 'EV', 'F9', 'G4', 'HA', 'MQ', 'NK', 'OH',
       'OO', 'VX', 'WN', 'YV', 'YX', 'AA', 'DL'], dtype=object)

In [7]:
df1['OP_CARRIER'].replace({
    'UA':'United Airlines',
    'AS':'Alaska Airlines',
    '9E':'Endeavor Air',
    'B6':'JetBlue Airways',
    'EV':'ExpressJet',
    'F9':'Frontier Airlines',
    'G4':'Allegiant Air',
    'HA':'Hawaiian Airlines',
    'MQ':'Envoy Air',
    'NK':'Spirit Airlines',
    'OH':'PSA Airlines',
    'OO':'SkyWest Airlines',
    'VX':'Virgin America',
    'WN':'Southwest Airlines',
    'YV':'Mesa Airline',
    'YX':'Republic Airways',
    'AA':'American Airlines',
    'DL':'Delta Airlines'
}, inplace=True)

In [8]:
df1.OP_CARRIER.nunique()

18

In [9]:
df1.OP_CARRIER.value_counts()

Southwest Airlines    1352552
Delta Airlines         949283
American Airlines      916818
SkyWest Airlines       774137
United Airlines        621565
Republic Airways       316090
JetBlue Airways        305010
Envoy Air              296001
PSA Airlines           278457
Endeavor Air           245917
Alaska Airlines        245761
Mesa Airline           215138
ExpressJet             202890
Spirit Airlines        176178
Frontier Airlines      120035
Allegiant Air           96221
Hawaiian Airlines       83723
Virgin America          17670
Name: OP_CARRIER, dtype: int64

### Unnamed: 27 COLUMN

This is column 27 which is completly empty

In [10]:
df1["Unnamed: 27"].nunique()

0

In [11]:
df1 = df1.drop(["Unnamed: 27"], axis=1)

### CANCELLED COLUMN

### According to the definition of delayed flights, a cancelled flight is not a delayed flight as it never left not arrived to its destination for whatever reason

In [12]:
df1.CANCELLED.nunique()

2

In [13]:
df1.CANCELLED.value_counts()

0.0    7096862
1.0     116584
Name: CANCELLED, dtype: int64

This is a binary column where number 1 equals a canceled flight, and a 0 not canceled, therefore we can drop the flights that were canceled, equivalent to 116584.

In [14]:
dummy = df1[(df1['CANCELLED'] == 1)]
print('dummy shape:', dummy.shape)
print('====================================')
print('dummy dataframe:')
dummy.head(3).append(dummy.tail(3))

dummy shape: (116584, 27)
dummy dataframe:


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
178,2018-01-01,United Airlines,2034,IAH,MFE,1440,NaN,NaN,NaN,NaN,NaN,NaN,1601,NaN,NaN,1.0,B,0.0,81.0,NaN,NaN,316.0,NaN,NaN,NaN,NaN,NaN
875,2018-01-01,United Airlines,864,LAS,SFO,1744,NaN,NaN,NaN,NaN,NaN,NaN,1926,NaN,NaN,1.0,A,0.0,102.0,NaN,NaN,414.0,NaN,NaN,NaN,NaN,NaN
1244,2018-01-01,United Airlines,488,MFE,IAH,1726,NaN,NaN,NaN,NaN,NaN,NaN,1844,NaN,NaN,1.0,B,0.0,78.0,NaN,NaN,316.0,NaN,NaN,NaN,NaN,NaN
7212759,2018-12-31,American Airlines,149,EGE,LAX,1444,NaN,NaN,NaN,NaN,NaN,NaN,1606,NaN,NaN,1.0,B,0.0,142.0,NaN,NaN,748.0,NaN,NaN,NaN,NaN,NaN
7212772,2018-12-31,American Airlines,165,DFW,EGE,845,836.0,-9.0,10.0,846.0,NaN,NaN,1019,NaN,NaN,1.0,B,0.0,154.0,NaN,NaN,721.0,NaN,NaN,NaN,NaN,NaN
7212773,2018-12-31,American Airlines,165,EGE,DFW,1059,NaN,NaN,NaN,NaN,NaN,NaN,1410,NaN,NaN,1.0,B,0.0,131.0,NaN,NaN,721.0,NaN,NaN,NaN,NaN,NaN


In [15]:
dummy.CANCELLED.nunique()

1

In [16]:
dummy.CANCELLED.value_counts()

1.0    116584
Name: CANCELLED, dtype: int64

In [17]:
df1 = df1[(df1['CANCELLED'] == 0)]
df1.head(2)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CANCELLED,CANCELLATION_CODE,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2018-01-01,United Airlines,2429,EWR,DEN,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,NaN,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,United Airlines,2427,LAS,SFO,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,NaN,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN


In [18]:
df1 = df1.drop(['CANCELLED'], axis = 1) 

In [19]:
df1.shape

(7096862, 26)

### CANCELLATION_CODE

In [20]:
df1.CANCELLATION_CODE.nunique()

0

In [21]:
df1.CANCELLATION_CODE.value_counts()

Series([], Name: CANCELLATION_CODE, dtype: int64)

In [22]:
df1.CANCELLATION_CODE.unique()

array([nan], dtype=object)

In [23]:
CC_A = df1['CANCELLATION_CODE'] == 'A'
CC_B = df1['CANCELLATION_CODE'] == 'B'
CC_C = df1['CANCELLATION_CODE'] == 'C'
CC_D = df1['CANCELLATION_CODE'] == 'D'

In [24]:
df1_CC_A = df1[CC_A]
df1_CC_B = df1[CC_B]
df1_CC_C = df1[CC_C]
df1_CC_D = df1[CC_D]
print('df1_CC_A shape:', df1_CC_A.shape)
print('df1_CC_B shape:', df1_CC_B.shape)
print('df1_CC_C shape:', df1_CC_C.shape)
print('df1_CC_D shape:', df1_CC_D.shape)

df1_CC_A shape: (0, 26)
df1_CC_B shape: (0, 26)
df1_CC_C shape: (0, 26)
df1_CC_D shape: (0, 26)


At this point, based on the previous tests it is safe to assume that the CANCELLATION_CODE column provided in this file is empty. We are going to drop the column

In [25]:
df1 = df1.drop(["CANCELLATION_CODE"], axis=1)

### ORIGIN & DEST

These two columns have the IATA airport codes for the origin and the destination. We will change it to a real city name to we can do EDA as per the city as well. 

We will be using and modifying an airports file from 2015 which contains most of the IATA codes

In [26]:
airports = pd.read_csv('C:\\Users\\udaya\\OneDrive\\Documents\\flight\\airports.csv')
airports.head(10)

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447
5,ACK,Nantucket Memorial Airport,Nantucket,MA,USA,41.25305,-70.06018
6,ACT,Waco Regional Airport,Waco,TX,USA,31.61129,-97.23052
7,ACV,Arcata Airport,Arcata/Eureka,CA,USA,40.97812,-124.10862
8,ACY,Atlantic City International Airport,Atlantic City,NJ,USA,39.45758,-74.57717
9,ADK,Adak Airport,Adak,AK,USA,51.87796,-176.64603


In [27]:
airports.shape

(322, 7)

In [28]:
len(airports.IATA_CODE.unique())

322

In [29]:
airport_IATA_CODE = list(airports['IATA_CODE'])

In [30]:
len(airport_IATA_CODE)

322

In [31]:
df1.ORIGIN.unique()

array(['EWR', 'LAS', 'SNA', 'RSW', 'ORD', 'IAH', 'DEN', 'SMF', 'RIC',
       'PDX', 'MCO', 'TYS', 'SFO', 'JAC', 'BOS', 'MSY', 'MIA', 'SEA',
       'SAT', 'SLC', 'RDU', 'FLL', 'IAD', 'DFW', 'ANC', 'MSP', 'ALB',
       'LAX', 'IND', 'SAN', 'BNA', 'BDL', 'ABQ', 'SAV', 'PHX', 'AUS',
       'PHL', 'SJC', 'ORF', 'DCA', 'LGA', 'BWI', 'PIT', 'OGG', 'CLE',
       'TPA', 'MYR', 'ROC', 'SJU', 'EGE', 'HNL', 'ONT', 'PBI', 'MKE',
       'HDN', 'JAX', 'OKC', 'ATL', 'SRQ', 'BZN', 'EUG', 'BOI', 'RNO',
       'GEG', 'TUS', 'LIH', 'KOA', 'ITO', 'PSP', 'DTW', 'GUC', 'OMA',
       'MTJ', 'CLT', 'CHS', 'MCI', 'BIL', 'CVG', 'CID', 'MEM', 'AVL',
       'BUF', 'GUM', 'JFK', 'ADQ', 'BET', 'SCC', 'BRW', 'FAI', 'JNU',
       'KTN', 'SIT', 'PSG', 'OME', 'OTZ', 'BUR', 'OAK', 'BLI', 'SBA',
       'STL', 'GFK', 'SYR', 'GSP', 'FSD', 'DSM', 'ILM', 'PWM', 'BIS',
       'JAN', 'GRB', 'OAJ', 'BTV', 'TLH', 'LAN', 'MSN', 'BMI', 'BGR',
       'ABY', 'MOT', 'DHN', 'LEX', 'MDT', 'CMH', 'FSM', 'HSV', 'FAR',
       'BQK', 'GSO',

In [32]:
df1_ORIGIN = df1.ORIGIN.unique()
df1_DEST = df1.DEST.unique()

In [33]:
print(type(df1_ORIGIN))
print(type(df1_DEST))

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [34]:
df1_ORIGIN = df1_ORIGIN.tolist()
df1_DEST = df1_DEST.tolist()

In [35]:
print(type(df1_ORIGIN))
print(type(df1_DEST))

<class 'list'>
<class 'list'>


In [36]:
print(len(df1_ORIGIN))
print(len(df1_DEST))

358
358


In [37]:
# differences between df1_ORIGIN and airport_IATA_CODE
difference_1 = [item for item in airport_IATA_CODE if item not in df1_ORIGIN]
difference_1

['CEC', 'CLD', 'DIK', 'ILG', 'UST']

In [38]:
difference_2 = [item for item in df1_ORIGIN if item not in airport_IATA_CODE]
difference_2

['SFB',
 'PIE',
 'BLV',
 'HGR',
 'RFD',
 'OWB',
 'HTS',
 'PGD',
 'USA',
 'LCK',
 'AZA',
 'OGD',
 'PVU',
 'YNG',
 'SCK',
 'SWO',
 'HVN',
 'STS',
 'IFP',
 'CKB',
 'CGI',
 'UIN',
 'PSM',
 'OGS',
 'LYH',
 'BFF',
 'LBF',
 'LBL',
 'FLO',
 'PGV',
 'LWB',
 'SHD',
 'SLN',
 'SPN',
 'BKG',
 'HHH',
 'PRC',
 'EAR',
 'DRT',
 'CYS',
 'ART']

In [39]:
len(difference_2)

41

In [40]:
difference_3 = [item for item in df1_DEST if item not in airport_IATA_CODE]
difference_4 = [item for item in airport_IATA_CODE if item not in df1_DEST]
print(type(difference_3))
print(type(difference_4))

<class 'list'>
<class 'list'>


In [41]:
# These are the 5 rows that will need to be eliminated from the original airport_IATA_CODE list/dataframe
print(difference_1)
print(difference_4)

['CEC', 'CLD', 'DIK', 'ILG', 'UST']
['CEC', 'CLD', 'DIK', 'ILG', 'UST']


In [42]:
# Now let's try to see if the differences between 

difference_2_3 = [item for item in difference_2 if item not in difference_3]
difference_2_3

[]

In [43]:
missing_airports = pd.DataFrame(difference_2, columns=['IATA_CODE'])
print('missing_airports type is:', type(missing_airports))
print('===============================================================')
print('Dataframe difference_2 shape:', missing_airports.shape)
print('===============================================================')
missing_airports.head(3).append(missing_airports.tail(3))

missing_airports type is: <class 'pandas.core.frame.DataFrame'>
Dataframe difference_2 shape: (41, 1)


,IATA_CODE
0,SFB
1,PIE
2,BLV
38,DRT
39,CYS
40,ART


In [44]:
missing_airports.to_csv('missing_airports.csv')

In [45]:
missing_airports = pd.read_excel('missing_airports_filled.xlsx', index_col=0)
print(missing_airports.shape)
missing_airports.head(3).append(missing_airports.tail(2))

(41, 7)


,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,SFB,NaN,NaN,NaN,NaN,NaN,NaN
1,PIE,NaN,NaN,NaN,NaN,NaN,NaN
2,BLV,NaN,NaN,NaN,NaN,NaN,NaN
39,CYS,NaN,NaN,NaN,NaN,NaN,NaN
40,ART,NaN,NaN,NaN,NaN,NaN,NaN


In [46]:
print(airports.shape)
airports.head(3)

(322, 7)


,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919


In [47]:
airports = airports[airports.IATA_CODE != 'CEC']
airports = airports[airports.IATA_CODE != 'CLD']
airports = airports[airports.IATA_CODE != 'DIK']
airports = airports[airports.IATA_CODE != 'ILG']
airports = airports[airports.IATA_CODE != 'UST']

In [48]:
airports.shape

(317, 7)

In [49]:
# Appending the missing_airports dataframe to the larger airports dataframe
AIRPORTS = airports.append(missing_airports, ignore_index=True)
AIRPORTS.shape

(358, 7)

In [50]:
# Now a quick visualization of the first and last 2 rows
AIRPORTS.head(2).append(AIRPORTS.tail(2))

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.4404
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.6819
356,CYS,NaN,NaN,NaN,NaN,NaN,NaN
357,ART,NaN,NaN,NaN,NaN,NaN,NaN


In [51]:
print('Number of cities with at least one airport:', AIRPORTS.CITY.nunique())
print('================================================')
print('Total number of airports:', AIRPORTS.IATA_CODE.nunique())
print('================================================')

Number of cities with at least one airport: 305
Total number of airports: 358


In [52]:
# Creating a dictionary and looking at its type for QC purposes
airport_dict = pd.Series(AIRPORTS.CITY.values, index=AIRPORTS.IATA_CODE).to_dict()
print(type(airport_dict))

<class 'dict'>


In [53]:
df1['ORIGIN'].replace(airport_dict, inplace=True)
df1['DEST'].replace(airport_dict, inplace=True)
df1.head(3).append(df1.tail(3))

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2018-01-01,United Airlines,2429,Newark,Denver,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,0.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,United Airlines,2427,Las Vegas,San Francisco,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,0.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,United Airlines,2426,Santa Ana,Denver,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,0.0,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN
7213443,2018-12-31,American Airlines,1817,Charlotte,Memphis,2015,2010.0,-5.0,36.0,2046.0,2114.0,4.0,2107,2118.0,11.0,0.0,112.0,128.0,88.0,511.0,NaN,NaN,NaN,NaN,NaN
7213444,2018-12-31,American Airlines,1818,Charlotte,Raleigh,1300,1323.0,23.0,11.0,1334.0,1400.0,4.0,1350,1404.0,14.0,0.0,50.0,41.0,26.0,130.0,NaN,NaN,NaN,NaN,NaN
7213445,2018-12-31,American Airlines,1818,Raleigh,Charlotte,1435,1443.0,8.0,8.0,1451.0,1535.0,7.0,1546,1542.0,-4.0,0.0,71.0,59.0,44.0,130.0,NaN,NaN,NaN,NaN,NaN


Successfully changed the IATA codes for the ORIGIN and DESTINATION columns.

In [54]:
print('Total number of departure/origin cities:', df1.ORIGIN.nunique())
print('==================================================')
print('Total number of arrivals/destinations cities:', df1.DEST.nunique())
print('==================================================')

Total number of departure/origin cities: 305
Total number of arrivals/destinations cities: 305


### DIVERTED COLUMN`

A diverted flight is one in which the plane lands on a different airport that scheduled. This is normally temporary and the airline has the responsibility of taking passengers to their final destination without any extra cost. Therefore this will be considered as a delayed flight.

In [55]:
df1.DIVERTED.value_counts()

0.0    7079005
1.0      17857
Name: DIVERTED, dtype: int64

It is clear that this is a binary feature, but we are not sure what either value means so we are going to analyze the entire column. 

Since the number of 1s is a lot less than 0s, we will create a seperate dataframe with only the columns where DIVERTED == 1

In [56]:
DIVERTED_1 = df1['DIVERTED']==1

In [57]:
df1_DIV = df1[DIVERTED_1]
print(df1_DIV.shape)

(17857, 25)


In [58]:
df1_DIV.head()

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,DIVERTED,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
1168,2018-01-01,United Airlines,555,San Francisco,San Diego,729,718.0,-11.0,17.0,735.0,1124.0,3.0,903,1127.0,NaN,1.0,94.0,NaN,NaN,447.0,NaN,NaN,NaN,NaN,NaN
1638,2018-01-01,Alaska Airlines,64,Petersburg,Wrangell,1514,1626.0,72.0,20.0,1646.0,NaN,NaN,1538,NaN,NaN,1.0,24.0,NaN,NaN,31.0,NaN,NaN,NaN,NaN,NaN
1643,2018-01-01,Alaska Airlines,65,Ketchikan,Wrangell,925,918.0,-7.0,26.0,944.0,NaN,NaN,1011,NaN,NaN,1.0,46.0,NaN,NaN,82.0,NaN,NaN,NaN,NaN,NaN
1683,2018-01-01,Alaska Airlines,120,Seattle,San Diego,550,544.0,-6.0,18.0,602.0,1046.0,3.0,840,1049.0,NaN,1.0,170.0,NaN,NaN,1050.0,NaN,NaN,NaN,NaN,NaN
1850,2018-01-01,Alaska Airlines,512,Seattle,San Diego,725,722.0,-3.0,16.0,738.0,1216.0,3.0,1017,1219.0,NaN,1.0,172.0,NaN,NaN,1050.0,NaN,NaN,NaN,NaN,NaN


In [59]:
df1_DIV.ARR_DELAY.nunique()

0

In [60]:
df1_DIV.ARR_DELAY.unique()

array([nan])

The last two command lines tell us that the ARR_DELAY, which refers to the total delay on arrival in minutes, are all equal to NaN when DIVERTED == 1. So basically there is no information to consider it as if it was delay or not. So we can either distribute these values into delayed and not delay with the same proportion currently present, or we can just drop them.

In [61]:
df1_DIV.AIR_TIME.nunique()

0

In [62]:
df1 = df1.drop(['DIVERTED'], axis=1)
df1.head(3)

,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,CARRIER_DELAY,WEATHER_DELAY,NAS_DELAY,SECURITY_DELAY,LATE_AIRCRAFT_DELAY
0,2018-01-01,United Airlines,2429,Newark,Denver,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,268.0,250.0,225.0,1605.0,NaN,NaN,NaN,NaN,NaN
1,2018-01-01,United Airlines,2427,Las Vegas,San Francisco,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,99.0,83.0,65.0,414.0,NaN,NaN,NaN,NaN,NaN
2,2018-01-01,United Airlines,2426,Santa Ana,Denver,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,134.0,126.0,106.0,846.0,NaN,NaN,NaN,NaN,NaN


In [63]:
df1.shape

(7096862, 24)

### DELAY REASONS COLUMN

In this section we will deal with 5 related columns:
* CARRIER_DELAY<br>
* WEATHER_DELAY<br>
* NAS_DELAY<br>
* SECURITY_DELAY<br>
* LATE_AIRCRAFT_DELAY

In [64]:
print(df1.CARRIER_DELAY.isna().sum())
print(df1.WEATHER_DELAY.isna().sum())
print(df1.NAS_DELAY.isna().sum())
print(df1.SECURITY_DELAY.isna().sum())
print(df1.LATE_AIRCRAFT_DELAY.isna().sum())

5744152
5744152
5744152
5744152
5744152


In [65]:
print("Percentage of valid data:", 100 - (df1.CARRIER_DELAY.isna().sum()*100/len(df1)))
print("Percentage of missing values:", (df1.CARRIER_DELAY.isna().sum()*100/len(df1)))

Percentage of valid data: 19.060677803795542
Percentage of missing values: 80.93932219620446


Drop all these columns because of the huge amounts of missing data

In [66]:
df1 = df1.drop(['CARRIER_DELAY', 'WEATHER_DELAY', 'NAS_DELAY', 'SECURITY_DELAY', 'LATE_AIRCRAFT_DELAY'], axis=1)
print('The current shape of df1 is:', df1.shape)
df1.head(3).append(df1.tail(3))

The current shape of df1 is: (7096862, 19)


,FL_DATE,OP_CARRIER,OP_CARRIER_FL_NUM,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,2429,Newark,Denver,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,2427,Las Vegas,San Francisco,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,United Airlines,2426,Santa Ana,Denver,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,134.0,126.0,106.0,846.0
7213443,2018-12-31,American Airlines,1817,Charlotte,Memphis,2015,2010.0,-5.0,36.0,2046.0,2114.0,4.0,2107,2118.0,11.0,112.0,128.0,88.0,511.0
7213444,2018-12-31,American Airlines,1818,Charlotte,Raleigh,1300,1323.0,23.0,11.0,1334.0,1400.0,4.0,1350,1404.0,14.0,50.0,41.0,26.0,130.0
7213445,2018-12-31,American Airlines,1818,Raleigh,Charlotte,1435,1443.0,8.0,8.0,1451.0,1535.0,7.0,1546,1542.0,-4.0,71.0,59.0,44.0,130.0


### OP_CARRIER_FL_NUM

Since we will not be using flight numbers for our predictinos, we drop the column

In [68]:
df1 = df1.drop(['OP_CARRIER_FL_NUM'], axis=1)
df1.head(3)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,Newark,Denver,1517,1512.0,-5.0,15.0,1527.0,1712.0,10.0,1745,1722.0,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,1115,1107.0,-8.0,11.0,1118.0,1223.0,7.0,1254,1230.0,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,United Airlines,Santa Ana,Denver,1335,1330.0,-5.0,15.0,1345.0,1631.0,5.0,1649,1636.0,-13.0,134.0,126.0,106.0,846.0


### DEP_TIME & AIR_TIME

After analyszing the data, we realized that the content of the DEP_TIME (departure time) and ARR_TIME(arrival time) variables can be a bit misleading since they do not contain dates.

For example, let's assume a schedule departure at 0h05 on the 1st of the month, but then the departure time says 23h50. What really happened here? is there a 23 hour and 45 minute delay? or did the flight leave 15 minutes earlier?

Now lets consider the DEP_DELAY and ARR_DELAY columns. Both of these are more informative because they give us directly the delay in minutes. So in case of the example above, the Departure Delay would be -15 min telling us that the flight left early. Hence, we will be dropping Departure and Arrival Time.

In [69]:
df1.drop(columns=['DEP_TIME', 'ARR_TIME'], inplace=True)
df1.head(2)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,Newark,Denver,1517,-5.0,15.0,1527.0,1712.0,10.0,1745,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,1115,-8.0,11.0,1118.0,1223.0,7.0,1254,-24.0,99.0,83.0,65.0,414.0


In [70]:
df1.to_csv('df1.csv')

# More Cleaning

## Dealing with Missing Values (NaN)

In [71]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7096862 entries, 0 to 7213445
Data columns (total 16 columns):
 #   Column               Dtype  
---  ------               -----  
 0   FL_DATE              object 
 1   OP_CARRIER           object 
 2   ORIGIN               object 
 3   DEST                 object 
 4   CRS_DEP_TIME         int64  
 5   DEP_DELAY            float64
 6   TAXI_OUT             float64
 7   WHEELS_OFF           float64
 8   WHEELS_ON            float64
 9   TAXI_IN              float64
 10  CRS_ARR_TIME         int64  
 11  ARR_DELAY            float64
 12  CRS_ELAPSED_TIME     float64
 13  ACTUAL_ELAPSED_TIME  float64
 14  AIR_TIME             float64
 15  DISTANCE             float64
dtypes: float64(10), int64(2), object(4)
memory usage: 920.5+ MB


In [72]:
df1.isna().sum()

FL_DATE                    0
OP_CARRIER                 0
ORIGIN                 46556
DEST                   46609
CRS_DEP_TIME               0
DEP_DELAY               4743
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON               2662
TAXI_IN                 2662
CRS_ARR_TIME               0
ARR_DELAY              20456
CRS_ELAPSED_TIME           7
ACTUAL_ELAPSED_TIME    17858
AIR_TIME               17858
DISTANCE                   0
dtype: int64

In [73]:
print('Number of total missing values:', df1.isna().sum().sum())
print('df1 shape:', df1.shape)

Number of total missing values: 159411
df1 shape: (7096862, 16)


There are quite a number of missing values, although compared to the size of the entire dataframe, it is just a minimal percentage considerable below 1%, so innprinciple we could drop these rows but before going down that route we will understand some of these missing values

In [74]:
df_NaN = df1.loc[df1.isnull().any(axis=1)]
df_NaN.head(20)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
1168,2018-01-01,United Airlines,San Francisco,San Diego,729,-11.0,17.0,735.0,1124.0,3.0,903,NaN,94.0,NaN,NaN,447.0
1638,2018-01-01,Alaska Airlines,Petersburg,Wrangell,1514,72.0,20.0,1646.0,NaN,NaN,1538,NaN,24.0,NaN,NaN,31.0
1643,2018-01-01,Alaska Airlines,Ketchikan,Wrangell,925,-7.0,26.0,944.0,NaN,NaN,1011,NaN,46.0,NaN,NaN,82.0
1683,2018-01-01,Alaska Airlines,Seattle,San Diego,550,-6.0,18.0,602.0,1046.0,3.0,840,NaN,170.0,NaN,NaN,1050.0
1850,2018-01-01,Alaska Airlines,Seattle,San Diego,725,-3.0,16.0,738.0,1216.0,3.0,1017,NaN,172.0,NaN,NaN,1050.0
1886,2018-01-01,Alaska Airlines,Portland,San Diego,625,6.0,17.0,648.0,1147.0,4.0,852,NaN,147.0,NaN,NaN,933.0
1958,2018-01-01,Alaska Airlines,Salt Lake City,San Diego,600,-6.0,31.0,625.0,1031.0,3.0,708,NaN,128.0,NaN,NaN,626.0
2000,2018-01-01,Alaska Airlines,Baltimore,San Diego,625,1.0,9.0,635.0,1111.0,4.0,905,NaN,340.0,NaN,NaN,2295.0
2102,2018-01-01,Endeavor Air,Cleveland,Detroit,630,-5.0,39.0,704.0,731.0,8.0,739,NaN,69.0,74.0,27.0,95.0
2124,2018-01-01,Endeavor Air,Dallas-Fort Worth,New York,900,NaN,16.0,916.0,1303.0,6.0,1334,-25.0,214.0,189.0,167.0,1391.0


In [75]:
df_NaN.shape

(109049, 16)

There are a total of 25,045 rows with missing values out of 7,096,862 rows of data

In [76]:
percentage_NaN = (len(df_NaN)*100)/len(df1)
print('Percentage of rows with at least 1 NaN value:', percentage_NaN)

Percentage of rows with at least 1 NaN value: 1.5365805337626686


### DEP_DELAY

The DEP_DELAY refers to the "Departure Delay", which is the difference between DEP_TIME (Actual Departure Time) and the SCH_DEP_TIME (Scheduled Departute time), therefore, if there are no delays for the departure, then the DEP_DELAY equals 0 (zero), and this is when it ends up being a NaN. This is following the definition of DEP_DELAY and by double checking the 20 rows display just above. So we will replace the NaN on this column by a 0.0 meaning no departure delay and this will help us lower this number from 4743 to 0. 

In [77]:
df1["DEP_DELAY"] = df1["DEP_DELAY"].fillna(0)
df1.isna().sum()

FL_DATE                    0
OP_CARRIER                 0
ORIGIN                 46556
DEST                   46609
CRS_DEP_TIME               0
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON               2662
TAXI_IN                 2662
CRS_ARR_TIME               0
ARR_DELAY              20456
CRS_ELAPSED_TIME           7
ACTUAL_ELAPSED_TIME    17858
AIR_TIME               17858
DISTANCE                   0
dtype: int64

### TAXI_IN

There is no way that to find out the values of the missing TAXI_IN. What I can do is replace the NaN by the mean or median.

In [78]:
df1['TAXI_IN'].fillna((df1['TAXI_IN'].mean()), inplace=True)
df_NaN['TAXI_IN'].fillna((df_NaN['TAXI_IN'].mean()), inplace=True)

In [79]:
df_NaN.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
1168,2018-01-01,United Airlines,San Francisco,San Diego,729,-11.0,17.0,735.0,1124.0,3.000000,903,NaN,94.0,NaN,NaN,447.0
1638,2018-01-01,Alaska Airlines,Petersburg,Wrangell,1514,72.0,20.0,1646.0,NaN,7.319748,1538,NaN,24.0,NaN,NaN,31.0
1643,2018-01-01,Alaska Airlines,Ketchikan,Wrangell,925,-7.0,26.0,944.0,NaN,7.319748,1011,NaN,46.0,NaN,NaN,82.0
1683,2018-01-01,Alaska Airlines,Seattle,San Diego,550,-6.0,18.0,602.0,1046.0,3.000000,840,NaN,170.0,NaN,NaN,1050.0
1850,2018-01-01,Alaska Airlines,Seattle,San Diego,725,-3.0,16.0,738.0,1216.0,3.000000,1017,NaN,172.0,NaN,NaN,1050.0


In [80]:
df1.TAXI_IN.isna().sum()

0

In [81]:
df1.isna().sum()

FL_DATE                    0
OP_CARRIER                 0
ORIGIN                 46556
DEST                   46609
CRS_DEP_TIME               0
DEP_DELAY                  0
TAXI_OUT                   0
WHEELS_OFF                 0
WHEELS_ON               2662
TAXI_IN                    0
CRS_ARR_TIME               0
ARR_DELAY              20456
CRS_ELAPSED_TIME           7
ACTUAL_ELAPSED_TIME    17858
AIR_TIME               17858
DISTANCE                   0
dtype: int64

In [82]:
df1 = df1.dropna()

In [83]:
df1.shape

(6992401, 16)

In [84]:
df1.isna().sum()

FL_DATE                0
OP_CARRIER             0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_DELAY              0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_DELAY              0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
dtype: int64

In [85]:
df1.head(2)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,Newark,Denver,1517,-5.0,15.0,1527.0,1712.0,10.0,1745,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,1115,-8.0,11.0,1118.0,1223.0,7.0,1254,-24.0,99.0,83.0,65.0,414.0


### Time Related Columns

Time is normally a categorical and having it in the current format will give us too many columns when the hot encode is applied to them, therefore we split the time into 4 quarters of the days meaning of 6 hours each.

In [86]:
df1['CRS_DEP_TIME'] = np.ceil(df1['CRS_DEP_TIME']/600).apply(int)
df1['WHEELS_OFF'] = np.ceil(df1['WHEELS_OFF']/600).apply(int) 
df1['WHEELS_ON'] = np.ceil(df1['WHEELS_ON']/600).apply(int)
df1['CRS_ARR_TIME'] = np.ceil(df1['CRS_ARR_TIME']/600).apply(int)

In [87]:
df1.head()

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE
0,2018-01-01,United Airlines,Newark,Denver,3,-5.0,15.0,3,3,10.0,3,-23.0,268.0,250.0,225.0,1605.0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,2,-8.0,11.0,2,3,7.0,3,-24.0,99.0,83.0,65.0,414.0
2,2018-01-01,United Airlines,Santa Ana,Denver,3,-5.0,15.0,3,3,5.0,3,-13.0,134.0,126.0,106.0,846.0
3,2018-01-01,United Airlines,Ft. Myers,Chicago,3,6.0,19.0,3,3,6.0,3,-2.0,190.0,182.0,157.0,1120.0
4,2018-01-01,United Airlines,Chicago,Albany,2,20.0,13.0,2,2,10.0,2,14.0,112.0,106.0,83.0,723.0


# Data Engineering

### Date Format

We will create a few columns which might be useful for EDA. The first will be the month and the second will be the day.

In [90]:
df1['DAY'] = pd.DatetimeIndex(df1['FL_DATE']).day
df1['MONTH'] = pd.DatetimeIndex(df1['FL_DATE']).month
df1.head(2)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DAY,MONTH
0,2018-01-01,United Airlines,Newark,Denver,3,-5.0,15.0,3,3,10.0,3,-23.0,268.0,250.0,225.0,1605.0,1,1
1,2018-01-01,United Airlines,Las Vegas,San Francisco,2,-8.0,11.0,2,3,7.0,3,-24.0,99.0,83.0,65.0,414.0,1,1


In [91]:
print('Maximum number of days in a month:', df1.DAY.nunique())
print('Total number of months:', df1.MONTH.nunique())

Maximum number of days in a month: 31
Total number of months: 12


For visual purposes we will be useng month values to analyze the best and the worst months to travel in terms of delays, we will convert the numerical month value into abbreviation. For the modeling phase this will have no impact.

In [93]:
import calendar
df1['MONTH_AB'] = df1['MONTH'].apply(lambda x: calendar.month_abbr[x])

In [94]:
df1.head(2)

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DAY,MONTH,MONTH_AB
0,2018-01-01,United Airlines,Newark,Denver,3,-5.0,15.0,3,3,10.0,3,-23.0,268.0,250.0,225.0,1605.0,1,1,Jan
1,2018-01-01,United Airlines,Las Vegas,San Francisco,2,-8.0,11.0,2,3,7.0,3,-24.0,99.0,83.0,65.0,414.0,1,1,Jan


Double check

In [95]:
print('Months present in dataframe:', df1.MONTH.unique())
print('Months present in dataframe:', df1.MONTH_AB.unique())

Months present in dataframe: [ 1  2  3  4  5  6  7  8  9 10 11 12]
Months present in dataframe: ['Jan' 'Feb' 'Mar' 'Apr' 'May' 'Jun' 'Jul' 'Aug' 'Sep' 'Oct' 'Nov' 'Dec']


### Binary Classification

This is a binary classification, which means that we will run out models with the target bwing a column called FLIGHT_STATUS. In this column there will only be two values, 0 for flights that arrive either earlier or on time and 1 for the flights which are delayed

In [96]:
status = []

for value in df1['ARR_DELAY']:
    if value < 0:
        status.append(0)
    else:
        status.append(1)
df1['FLIGHT_STATUS'] = status
df1.head(2)    

,FL_DATE,OP_CARRIER,ORIGIN,DEST,CRS_DEP_TIME,DEP_DELAY,TAXI_OUT,WHEELS_OFF,WHEELS_ON,TAXI_IN,CRS_ARR_TIME,ARR_DELAY,CRS_ELAPSED_TIME,ACTUAL_ELAPSED_TIME,AIR_TIME,DISTANCE,DAY,MONTH,MONTH_AB,FLIGHT_STATUS
0,2018-01-01,United Airlines,Newark,Denver,3,-5.0,15.0,3,3,10.0,3,-23.0,268.0,250.0,225.0,1605.0,1,1,Jan,0
1,2018-01-01,United Airlines,Las Vegas,San Francisco,2,-8.0,11.0,2,3,7.0,3,-24.0,99.0,83.0,65.0,414.0,1,1,Jan,0


In [97]:
df1.FLIGHT_STATUS.value_counts(normalize=True)

0    0.625075
1    0.374925
Name: FLIGHT_STATUS, dtype: float64

This means that 62.48% of the flights have no delays, but that can be that they arrived either early or on time, and 37.52% of the flight were delayed. Quite a significant number if you think about it. 

In [98]:
df1.to_csv('df1.csv')

In [99]:
df1.isna().sum()

FL_DATE                0
OP_CARRIER             0
ORIGIN                 0
DEST                   0
CRS_DEP_TIME           0
DEP_DELAY              0
TAXI_OUT               0
WHEELS_OFF             0
WHEELS_ON              0
TAXI_IN                0
CRS_ARR_TIME           0
ARR_DELAY              0
CRS_ELAPSED_TIME       0
ACTUAL_ELAPSED_TIME    0
AIR_TIME               0
DISTANCE               0
DAY                    0
MONTH                  0
MONTH_AB               0
FLIGHT_STATUS          0
dtype: int64